# Drug Transport across a Virtual Skin Membrane:  Extended version
[Previous Version](SkinDiffusion.ipybnd)

## 1. Setup

### Initialize UG4 (for 2D and standard algebra)

In [1]:
InitUG(2, AlgebraType("CPU", 1));
ug_load_script("ug_util.lua")
ug_load_script("util/refinement_util.lua")

* Initializing: paths... done, bridge... done, plugins... fail                 *


### Create Domain

In [2]:
requiredSubsets = {"LIP", "COR", "BOTTOM_SC", "TOP_SC"}
gridName = "skin2d-aniso.ugx"
numRefs = 2

In [3]:
dom = util.CreateDomain(gridName, numRefs, requiredSubsets)

Loading Domain skin2d-aniso.ugx ... done.
Performing integrity check on domain ... done.
Refining(2): 1 2 done.


### Create Approximation space

In [4]:
approxSpaceDesc = { fct = "u", type = "Lagrange", order = 1 }

In [5]:
approxSpace = ApproximationSpace(dom)
approxSpace:add_fct(approxSpaceDesc.fct, approxSpaceDesc.type, approxSpaceDesc.order)
approxSpace:init_levels()
approxSpace:init_top_surface()
print("Approximation space:")
approxSpace:print_statistic()

Approximation space:
| ----------------------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                               |
|  Algebra: Block 1 (divide by 1 for #Index)                                                |
|                                                                                           |
|    GridLevel   |       Domain |       0: LIP |       1: COR | 2: BOTTOM_SC |    3: TOP_SC |
| ----------------------------------------------------------------------------------------- |
| (lev,    0)    |          680 |           32 |          608 |           20 |           20 |
| (lev,    1)    |         2613 |          783 |         1752 |           39 |           39 |
| (lev,    2)    |        10241 |         4367 |         5720 |           77 |           77 |
| (lev,    0, g) |          680 |           32 |          608 |           20 |           20 |
| (lev,    1, g) |         2613 |      

### Create a convection-diffusion-equation
Define model parameter

In [6]:
K={
    ["LIP"] = 1.0, ["COR"] = 1.0,
}

D={
     ["LIP"] = 1, ["COR"] = 0.01, 
}


In [7]:
elemDisc ={}

elemDisc["COR"] = ConvectionDiffusion("u", "COR", "fv1")
elemDisc["COR"]:set_diffusion(K["COR"]*D["COR"])
elemDisc["COR"]:set_mass_scale(K["COR"])

elemDisc["LIP"] = ConvectionDiffusion("u", "LIP", "fv1")
elemDisc["LIP"]:set_diffusion(K["LIP"]*D["LIP"])
elemDisc["LIP"]:set_mass_scale(K["LIP"])

In [8]:
dirichletBnd = DirichletBoundary()
dirichletBnd:add(1.0, "u", "TOP_SC")
dirichletBnd:add(0.0, "u", "BOTTOM_SC")

In [9]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc["LIP"])
domainDisc:add(elemDisc["COR"])
domainDisc:add(dirichletBnd)

## 2. Steady state problem
Flux is computed from steady state. Since configuration of a multigrid solver is somewhat tricky, we use an LU decomposition here:

In [10]:
A = AssembledLinearOperator(domainDisc)
u = GridFunction(approxSpace)
b = GridFunction(approxSpace)
u:set(0.0)


domainDisc:assemble_linear(A, b)
domainDisc:adjust_solution(u)

myLinearSolver =LU()
myLinearSolver:init(A, u)
myLinearSolver:apply(u, b)

Compute $J_\infty=J(t=\infty)$ for
$$ J(t)=\frac{1}{|\Gamma|}\int_\Gamma (-KD \nabla u(t,x)) \cdot \vec n dA$$

In [11]:
area=Integral(1.0, u, "BOTTOM_SC")
print("Surface area [um^2]:")
print(area)

surfaceFlux = {}
surfaceFlux["BOT"] = K["LIP"]*D["LIP"]*IntegrateNormalGradientOnManifold(u, "u", "BOTTOM_SC", "LIP")
surfaceFlux["TOP"] = K["LIP"]*D["LIP"]*IntegrateNormalGradientOnManifold(u, "u", "TOP_SC", "LIP")
print("Surface fluxes [kg/s]:")
print(surfaceFlux["TOP"])
print(surfaceFlux["BOT"])

print("Normalized Fluxes [kg / (mu^2 * s)]:")
print(surfaceFlux["TOP"]/area)
print(surfaceFlux["BOT"]/area)

Jref = 0.05681818181818

print("Non Fluxes [kg / (mu^2 * s)]:")
print(surfaceFlux["TOP"]/(area*Jref))
print(surfaceFlux["BOT"]/(area*Jref))

Surface area [um^2]:
30.1
Surface fluxes [kg/s]:
-0.021331943805368
0.021331943805564
Normalized Fluxes [kg / (mu^2 * s)]:
-0.00070870245200557
0.0007087024520121
Non Fluxes [kg / (mu^2 * s)]:
-0.012473163155299
0.012473163155413


## 3. Transient problem

After each time-step, we execute a a callback function `MyPostProcess`. In this function, print the solution and compute
$$
m(t_k):= \int_0^{t_k} J(s) \, ds \approx \sum_{i=1}^k(t_{i}- t_{i-1}) \frac{J(t_{i-1}) +J(t_i)}{2} 
$$
using the trapeziod rule. Moreover, we also compute the lag time $\tau$ from $m(t_k) = J_\infty(t_k - \tau)$.


In [12]:
-- auxiliary variables
-- for output 
out=VTKOutput()

-- for book-keeping
tOld = 0.0
jOld = 0.0
mOld = 0.0


function MyPostProcess(u, step, time)
  
  -- 1) Print solution to file.
  out:print("vtk/SkinDiffusionWithLagtime", u, step, time)
  
  -- 2) Compute fluxes.
  local gradFlux={}
  gradFlux["BOT"] = IntegrateNormalGradientOnManifold(u, "u", "BOTTOM_SC", "LIP")
  gradFlux["TOP"] = IntegrateNormalGradientOnManifold(u, "u", "TOP_SC", "LIP")
  
  local jTOP = K["LIP"]*D["LIP"]*gradFlux["TOP"]
  local jBOT = K["LIP"]*D["LIP"]*gradFlux["BOT"]
  print ("flux_top (\t"..time.."\t)=\t"..jTOP)
  print ("flux_bot (\t"..time.."\t)=\t"..jBOT)
  
  -- 3) Compute mass.
  local dt = time - tOld
  local mass = mOld + (time - tOld)*(jBOT + jOld)/2.0
  print ("mass_bot (\t"..time.."\t)=\t"..mass)
  
  -- 4) Compute lag time.
  print ("tlag=".. time - mass/jBOT )
  
  -- 5) Updates
  tOld = time
  jOld = jBOT
  mOld = mass

end

### Solve transient problem
For the purpose of illustration, we solve using `SolveNonlinearTimeProblem`:

 * First, we create a non-linear solver:

In [13]:
local solverDesc = {

    type = "newton",
    linSolver = myLinearSolver,
}

nlsolver = util.solver.CreateSolver(solverDesc)

* Set initial value  

In [14]:
u:set(0.0)

* Execute time stepping loop w/ fixed time-step 

In [15]:
local startTime = 0.0
local endTime = 500.0
local dt=25.0
local dtMin=2.5
util.SolveNonlinearTimeProblem(u, domainDisc, nlsolver, MyPostProcess, "vtk/skin",
                            "ImplEuler", 1, startTime, endTime, dt, dtMin); 

SolveNonlinearTimeProblem, Newton Solver setup:
NewtonSolver
 LinearSolver: 
 | LU Decomposition: Direct Solver for Linear Equation Systems.
 |  Minimum Entries for Sparse LU: 4000
 ConvergenceCheck: StdConvCheck( max steps = 100, min defect = 1e-12, relative reduction = 1e-06)
 LineSearch:  not set.

>> Writing start values
flux_top (	0	)=	0
flux_bot (	0	)=	0
mass_bot (	0	)=	0
tlag=nan
++++++ TIMESTEP 1 BEGIN (current time: 0) ++++++
++++++ Time step size: 25

   # ########     Newton Solver     #######################
   # ########  (Linear Solver: LU)  #######################
   #   Iter      Defect         Rate 
   #    0:    7.421977e+03      -------
   #    1:    3.611000e-12    4.865281e-16
   # Relative reduction 1.000000e-06 reached after 1 steps.
   # Average reduction over 1 steps: 4.865281e-16
   # #####  Iteration converged  #####

flux_top (	25	)=	-0.72291523102227
flux_bot (	25	)=	2.4713227150495e-13
mass_bot (	25	)=	3.0891533938119e-12
tlag=12.5
++++++ TIMESTEP 1 END   